In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


In [2]:
import numpy
import tflearn
import tensorflow
import random
import json

C:\Users\dell\.conda\envs\chatbot\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dell\.conda\envs\chatbot\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dell\.conda\envs\chatbot\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dell\.conda\envs\chatbot\lib\site-packages\tensorflow\python\framework

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [3]:
import pickle

In [4]:

with open('intents.json') as file:
    data = json.load(file)

In [5]:
print(data)

{'intents': [{'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'coaching', 'patterns': ['IAS coaching', 'UPSC coaching', 'SSC', 'State Level Exam', 'IIT', 'JEE', 'Bank', 'Banking', 'Java', 'C', 'C++', 'CA', 'PCS', 'Machine learning and Artificial intelligence', 'Deep learning '], 'responses': ['Yes , classes are available , which medium do you prefer ? Hindi or English?', 'Which medium do you prefer? Hindi or English']}, {'tag': 'query1', 'patterns': ['Hello', 'Hi there', 'Hi', 'Heya', 'How are you', 'Good morning', 'Good day'], 'responses': ['For which course are you planning ?', 'Tell us the course you are interested in please .']}, {'tag': 'query2', 'patterns': ['Hindi', 'English', 'Telugu'], 'responses': ['Looking for onli

In [5]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output,doc_x,doc_y = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

In [6]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [8]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [9]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [10]:
training = numpy.array(training)
output = numpy.array(output)

In [11]:
with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)



In [12]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\dell\Desktop\Programs\Machine learning\chatbot\model.tflearn


In [13]:
print(net.shape)

(?, 12)


In [14]:
# model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
 #model.save("model.tflearn")

In [1]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [17]:
def chat():
    print("Hi , Team Meritroad is here for your help (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

In [21]:
from flask import Flask, request, render_template

app = Flask(__name__)

model.load("model.tflearn")

@app.route('/result', methods = ['POST']) 
def chat():
    print("Hi , Team Meritroad is here for your help (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))
        return render_template('try2.html', prediction_text='Sales should be $ {}'.format(output))
    
if __name__ == "__main__":
    app.run(debug=True)

AttributeError: 'NoneType' object has no attribute 'load'

In [18]:
chat()

Hi , Team Meritroad is here for your help (type quit to stop)!
You: Hello
For which course are you planning ?
You: I am planning for PCS course
Which medium do you prefer? Hindi or English
You: English
Would you like to go for online or offline classes?
You: Online
You are a working professional or a student?
You: Student
What is the best time to contact you? 9AM to 6PM , after 6PM or anyother time?
You: 10 AM
We would like to know when do you want to join any course , which is this month/next month/next year or not sure?
You: This Month
Please provide us with your credentials : Name - _______ , Email - ________, Contact no.- _________ .(In this format) 
You: Kanishk Gupta , gkanishk67@gmail.com , 8303557273
Thanks for your corporation , our customer excecutive will get to you in no time . (type quit to end the conversation)
You: quit
